<a href="https://colab.research.google.com/github/nhdchicken/nhd-colab/blob/master/notebooks/inference/maskrcnnButtonsInferenceTF2.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mask R-CNN Demo on Buttons dataset**


> A quick intro to using the pre-trained model to detect the UI buttons.





This experiment is performed on colab so mount the nhdchicken drive.

In [ ]:
%%bash
if [  -d "/content/nhd-colab" ]
then
   echo "Environment already initialized"
elif [ -d "/content" ] && [ ! -d "/content/nhd-colab" ]
then
    cd /content || exit 1;
    echo "Installing https://github.com/nhdchicken/nhd-colab.git"
    git clone --recurse-submodules https://github.com/nhdchicken/nhd-colab.git || exit 1;
    cd nhd-colab || exit 1;
else
    echo "Not running in Colab - going to root of repos"
    cd `git rev-parse --show-toplevel` || exit 1;
fi
pwd
pip install utils/nhdcolab/ > /dev/null 2>&1 || exit 1;
nhdcolab > /dev/null 2>&1 || exit 1;
echo "Great Success!"

In [ ]:
!nhdcolab init mp-mask-rcnn

In [ ]:
from nhdcolab.environment import NHDEnvironment
NHD_ENV = NHDEnvironment(gdrive_mount=True)

Move to Mask_RCNN directory on the drive

In [ ]:
cd mask-rcnn/matterport/

Import the required libraries

In [ ]:
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import mrcnn
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN
import numpy as np
from numpy import zeros
from numpy import asarray
import imutils
import random
import cv2
import os
from keras.models import load_model
%matplotlib inline
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

We create a myMaskRCNNConfig class for training on the buttons dataset. It is derived from the base Mask R-CNN Config class and overrides some values.

In [ ]:
class myMaskRCNNConfig(Config):
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes (we would normally add +1 for the background)
     # kangaroo + BG
    NUM_CLASSES = 1+2
   
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 131
    
    # Learning rate
    LEARNING_RATE=0.003
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10

Create an instance of the myMaskRCNNConfig class

In [ ]:
config = myMaskRCNNConfig()
config.display()

Detecting objects in the image with masks and bounding box from the trained model and displaying the results



In [ ]:
class_names=['BG','button', 'button_off']
model = modellib.MaskRCNN(mode="inference", config=config, model_dir='./')
model_path = '..//..//..//drive//My Drive//models//mask-rcnn//maskrcnn_Buttons_TF2.2Epoch25Lr0.03.h5'
# loading the trained weights o the custom dataset
model.load_weights(model_path, by_name=True)
img = load_img("..//..//..//drive//My Drive//Buttons//sample-images/EnableDolby__HP__2YConfDAHT__1024x768.png")
img = img_to_array(img)

results= model.detect([img],verbose=1)
# Display results

r = results[0]
visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], 
                            title="Predictions")
